# Midterm notebook

This is the main notebook for the midterm submission from RWS group 3

## Table of contents
1. Analysis of incident data
2. The travel time of the road sections
3. Stochastic Gradient Descent(SGD) Optimizer

In [27]:
## Importing all neccesary modules
%matplotlib inline
import geopandas as gpd
import folium 
import json
import pandas as pd
import numpy as np

from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import branca.colormap as cm
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import pickle
import networkx as nx

## 3. Stochastic Gradient Descent(SGD) Optimizer

Stochastic Gradient Descent is an iterative optimization algorithm used to find the minimum of a cost or loss function. In this context, the cost function represents the total traveling time of inspectors to accidents. Explanations of the model are given as follows:

Input:
- inspectors: An array of coordinates representing the initial positions of inspectors.
- accidents: An array of coordinates representing the locations with high possibility of accidents.
- max_travel_time: A constant value representing the maximum allowed traveling time for an individual inspector.
- learning_rate: A hyperparameter controlling the step size during parameter updates in the optimization process.
- momentum: A hyperparameter representing the momentum term used in SGD with momentum.
- max_iterations: The number of iterations for the optimization process.

Output:
- optimized_inspectors: An array of coordinates representing the final positions of inspectors after the optimization process.

Optimization Objective:
- The optimization objective is to find the optimal positions for inspectors such that the total traveling time is minimized.
- The objective function minimizes the total traveling time while ensuring that no inspector's traveling time exceeds max_travel_time.
- The objective function is implicitly defined by the nearest_neighbor function, which assigns inspectors to the nearest accidents, and the subsequent update process.

In [ ]:
#import network X
G = pickle.load(open('NetworkX_graph.pickle', 'rb'))

Still there are some problems when computing the travel time, our group will continue to work on resolving these issues.

In [ ]:
# travel_time function
def travel_time_func(point1, point2):
    # This function should use the information given in network X to return the travel time between two points

    return 111

In [ ]:
# Define a function to find the nearest inspector-accident pair
def nearest_neighbor(available_inspectors, accidents, travel_time_func):
    min_travel_time = float('inf')
    nearest_inspector = None
    nearest_accident = None
    
    # Iterate over available inspectors and accidents to find the pair with the minimum travel time
    for inspector in available_inspectors:
        for accident in accidents:
            travel_time = travel_time_func(inspector, accident)
            if travel_time < min_travel_time:
                min_travel_time = travel_time
                nearest_inspector = inspector
                nearest_accident = accident
    
    return nearest_inspector, nearest_accident

# Define a function to calculate the total traveling time
def total_traveling_time(inspectors, accidents, travel_time_func):
    return sum(travel_time_func(inspector, accident) for inspector, accident in zip(inspectors, accidents))

# Define an optimization function using Stochastic Gradient Descent (SGD) with momentum
def sgd_with_momentum(learning_rate, momentum, max_iterations, inspectors, accidents, max_travel_time, travel_time_func):
    num_inspectors = len(inspectors)
    
    velocity = np.zeros_like(inspectors)
    
    # Iterate for a specified number of optimization iterations
    for _ in range(max_iterations):
        np.random.shuffle(inspectors)
        
        # Iterate over inspectors
        for i in range(num_inspectors):
            inspector, accident = nearest_neighbor(inspectors[i:], accidents, travel_time_func)
            current_travel_time = travel_time_func(inspector, accident)
            
            # Check if the current travel time exceeds the maximum allowed time
            if current_travel_time <= max_travel_time:
                continue
            
            # Calculate the gradient and update the inspector's position with momentum
            gradient = np.array(travel_time_func(inspector, accident)) * (inspector - accident) / current_travel_time
            velocity[i] = momentum * velocity[i] - learning_rate * gradient
            inspectors[i] += velocity[i]
    
    return inspectors



Besides, other variables are also not determined yet, such as the learning rate, interation, etc. Those types of variables can only be determined by running the model and comparing the results.

In [ ]:
# Other variables:
num_inspectors = 40
num_accidents = 25
max_travel_time = 14 #min
learning_rate = 0.1
momentum = 0.9
max_iterations = 1000


For the initial placement of inspectors, randomness is allowed, but it's crucial to ensure that each assigned position has only one inspector. Additionally, it's assumed that all inspectors are positioned on the highway, and their locations are not entirely arbitrary. We have a matrix containing all potential inspector positions, which correspond to the recorded locations of road sections in the provided dataset.

As for the accident positions, they represent locations (coordinates) obtained after applying a filtering process to identify areas with a higher likelihood of accidents. These part of work still require additional time and effort to be completed.

In [ ]:
# Define all possible inspector locations as an array
possible_inspector_locations = [[200,200],[300,400]]

# Initialize inspector positions without duplicates
inspectors = np.random.choice(possible_inspector_locations, num_inspectors, replace=False)

# Accident positions
accidents = [[100,200],[400,600]]

# Optimize inspector positions using SGD with momentum
optimized_inspectors = sgd_with_momentum(learning_rate, momentum, max_iterations, inspectors, accidents, max_travel_time, travel_time_func)

# Print the optimized inspector positions and total traveling time
print("Optimized Inspector Positions:", optimized_inspectors)
print("Total Traveling Time:", total_traveling_time(optimized_inspectors, accidents, travel_time_func))